In [1]:
import time,yahoo_fin,collections,csv,ftplib,io,mysql.connector,csv
from datetime import datetime as dt,time as t
import pandas as pd
from mysql.connector import Error
from sqlalchemy import create_engine
from IPython.display import display, HTML

NL="\n"

In [ ]:
#Setup MySQL database connection. input your credentials next to the correct variable. 
auth_plugin='mysql_native_password' 
host="127.0.0.1"
user="root"
passwd="Matt1994@)!^"
database="stonks_data"

#Dict. to be passed through function later. 
sql = {"a_p": auth_plugin, 
"h": host,
"u":user,
"p":passwd,
"db":database }

#Define functions to connect to SQL in other functions. 
def cursor(mydb):
    mycursor = mydb.cursor(buffered=True)
    return (mycursor)

def connect(a,h,u,p,db):
    mydb = mysql.connector.connect(auth_plugin=a,host=h,user=u,passwd=p,database=db)
    return (mydb)

#Tests that your connection is active. Should return "True".
mydb=connect(auth_plugin, host, user, passwd, database)
mycursor = cursor(mydb)
mydb.is_connected()

engine = create_engine("mysql+mysqlconnector://root:Matt1994@)!^@127.0.0.1/stonks_data")
con = engine.connect()

In [3]:
#Defining Functions

#insert_3 - 3.0 version of insert. Inserts a value into the database. Handled SQL errors if returned. Opens connection, runs insert statement, closes connection if successsful.

def insert_3(table,ticker, o={}, k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    t = get_SID(ticker,sql)
    d = assign_PID(sql)
    try:
        s = ", "
        command  = ("""INSERT INTO {} VALUES ({},{},"{}",{},{},"{}")""").format(table,str(d),str(t),o.get("buy_date"),str(o.get("shares")),str(o.get("cost")),o.get("portfolio"))
        print(command)
        mycursor.execute(command)
        mydb.commit()
        mycursor.close()
        global InsertError
        InsertError= False

    except mysql.connector.Error as e:
        sql_error=True
        print("Failed to insert record into table {}".format(e))
        InsertError = True
        return(InsertError)
    finally:
        if (mydb.is_connected()):
            mydb.close()
            
#update_sql - modifies existing database entries. Used for updating the number of shares owned, puchase date, etc.
def update_sql(table, p, o = {}, k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    command  = ("""UPDATE owned SET buy_date="{}", shares='{}', cost='{}' WHERE purchase_id={}""".format(o.get("buy_date"),o.get("shares"),o.get("cost"),p))
    try:
        mycursor.execute(command)
        mydb.commit()
        mycursor.close()
    except mysql.connector.Error as e:
        sql_error=True
        print("Failed to update record {}".format(e))

    finally:
        if (mydb.is_connected()):
            mydb.close()
            


In [1]:
#Defining Functions
#insert_2 legacy insert function. Still used in certain places in program. Lacks error hanndling. 

def insert_2(table, values, k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    try:
        command  = ("INSERT INTO "+table+" VALUES "+ str(values))
        mycursor.execute(command)
        mydb.commit()
        print("Record {} inserted successfully into {} table".format(values,table))
#         mycursor.close()
        global InsertError
        InsertError = False
    except mysql.connector.Error as e:
        print("Failed to insert record into table {}".format(e))
        InsertError = True
        
#no_stock: Used to add a new stock to the database. Checks to ensure that the entry does not exist, then gets input from the user to add to database. Includes data validation
#ensure that the correct type of data is added to the database. Formats input. Then commites to database. 
def no_stock(t,k={}):
    i = True
    while i != "NO":
        x=''
        while x =='':
            x = input("What is the name of the company?\n")
            name = x
            x = input("What index is it listed on?\n").upper()
            index = x
        mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
        mycursor = cursor(mydb)
        mycursor.execute("SELECT * FROM stock_main;")
        c = mycursor.fetchall()
        Stock_index=(len(c))
        Stock_index = Stock_index+1
        print("Do you want to commit these changes?")
        print(NL,Stock_index, t,name,index)
        q = input().upper()
        if q == "YES":
            insert_2("stock_main",(str(Stock_index),t,name,index),sql)
            break
        elif q == "NO":
            print("No changes were made. Try again?")
            i = input().upper()
            if i == "YES"or"NO":
                continue
            else:
                break
        else:
            print("""Please Enter "Yes" or "No".""")

In [5]:
#Defining Functions
#ud_date - updates the date value of an entry in the database with input from user. Includes data validation. 

def ud_date():
    x = True
    while x != '':
        x = input("What was the purchase date? (YY/MM/DD): ")
        if format_check(x) is True:
            return x
        else:
            print("Incorrect date format. Please enter date as YY/MM/DD")
#format_check used in ud_date to ensure that the correct data format is input by the user. 

def format_check(x):
    try:
        dt.strptime(x,'%y/%m/%d')
        return True
    except ValueError:
        return False

In [6]:
#Defining Functions
#ud_share - updates the "Shares" value in the database. Includes data vaildation. 

def ud_share():
    x = True
    while x != '':
        x = input("Number of shares purchased?: ")
        if int_check(x) is not False:
            return x
        else:
            print ("Please enter an number.")
#ud_cose - updates cost value in the database. Includes data validation      
def ud_cost():
    x=True
    while x !='':
        x = input("What is the cost per share?: ")
        if int_check(x) is not False:
            return x
        else:
            print("Please enter a number.")
#ud_port - Updates the portfolio value in the database. Includes data validation.           
def ud_port():
    x=True
    while x != "":
        x = input("Which portfolio is this stock in?\n1) Vanguard\n2) Robinhood\n")
        if x =="1":
            x = "Vanguard"
            return x
        elif x == "2":
            x = "Robinhood"
            return x
        else:
            print("Please either 1 or 2.")
 #int_check - checks to ensure that an integer is provided from user input. Used throughout.                  
def int_check(x):
    try:
        float(x)
    except ValueError:
        return False
    

In [7]:
#Defining Functions

#get_sid  - (Get Stock ID) Checks the database to see if there is a value for the StockID. If none, prompts user if they would like to add a new stock
# If value is found, returns the SID of the requested stock. includes SQL error handling. 

def get_SID(t,k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    try:
        command  = ("SELECT stock_id FROM stock_main WHERE ticker = '"+str(t)+"'")
        mycursor.execute(command)
        x = mycursor.fetchone()
        if x is None:
            while True:
                u = input("\nThat ticker is not listed in the database, do you want to \n manually add it?\n").upper()
                if u == "NO":
                    break
                elif u == "YES":
                    no_stock(str(t),sql)
                    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
                    mycursor = cursor(mydb)
                    command  = ("SELECT stock_id FROM stock_main WHERE ticker = '"+str(t)+"'")
                    mycursor.execute(command)
                    x = mycursor.fetchone()
                    y = int(x[0])
                    return y
                    break
                else:
                    print("""Please enter "Yes" or "No".""")
        else:
            y = int(x[0])
            return y
        mycursor.close()
    except mysql.connector.Error as e:
        sql_error=True
        print("An error occured: {}".format(e))
    finally:
        if (mydb.is_connected()):
            mydb.close()

#get_ticker = checks the database to see if the requested ticker is stored in the database". Includes SQL error handling. 
def get_ticker(t,k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    try:
        command  = ("SELECT ticker FROM stock_main WHERE stock_id = '"+str(t)+"'")
        mycursor.execute(command)
        x = mycursor.fetchone()
        if x is None:
            print("That Ticker is not listed in the database.")
            pass
        else:
            y = x[0]
            return(y)
        mycursor.close()
    except mysql.connector.Error as e:
        print("An error occured: {}".format(e))
    finally:
        if (mydb.is_connected()):
            mydb.close()


In [8]:
#Defining functions
#update_df - updates the temporary dataframe which holds the information to be queried against the database in other functions. 
def update_df(a_dict={}):
    quack = pd.read_sql('SELECT * FROM owned', mydb)
    for k in a_dict:
        x = get_SID(k,sql)
        if x is None:
            input("There is no stock ID for ticker: {}. Press any key to continue".format(k))
            pass
        else:
            quack = quack.append({'stock_id' : x,'buy_date':a_dict[k]['buy_date'],
                                    'shares':a_dict[k]['shares'],
                                   'cost': a_dict[k]['cost']},ignore_index=True)
    return(quack)

In [9]:
#Defining Functions
#update_owned - function that combines multiple functions from above. Uses the user input to builld the dictionary that is used to pass variables through
#to the SQL statement to the database. 

def update_owned(input_,o={}):
    print("This is the current data: \n")
    s = get_SID(input_,sql)
    show_partial("stock_id",s,"owned",sql)
    q= input("Which do you want to update? (Enter the Purchase ID)\nPress Enter to go back.\n")
    p=False
    while p != '':
        y = {}
        print(NL)
        y.update({'buy_date': ud_date()})
        y.update({'shares' : ud_share()})
        y.update({'cost': ud_cost()})
        y.update({'portfolio':ud_port()})
        print(NL,y)
        p = input("\nDo you want to commit these changes?: ").upper()
        if p == "YES":
            o.update({input_:y})
            update_sql("owned",q,y, sql)
            print(NL,"{} updated successfully!".format(input_))
            input("\nPress enter to continue...\n")
            break
        else:
            print(NL,"No changes were made.")
            input("\nPress enter to continue...\n")
            break
    

In [10]:
#Defining Functions
#df_insert_a -creates a SQL Insert statement with APPEND functionality. Converts a python dataframe to Sql table. 
def df_insert_a(df):
    mydb=connect(auth_plugin, host, user, passwd, database)
    x = "owned"
    df.to_sql(name = x, con = con, if_exists = 'append', chunksize = 1000,index=False, method=None)
    
#df_insert_r - creates a SQL Insert Statement with REPLACE functionality. Converts a python dataframe to sql table. 
def df_insert_r(df):
    mydb=connect(auth_plugin, host, user, passwd, database)
    x = "owned"
    df.to_sql(name = x, con = con, if_exists = 'replace', chunksize = 1000,index=False,method=None)
    
#drop_row - deletes a row from the database per user input. includes SQL error handling. 
def drop_row(t,k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    x = get_SID(t,sql)
    try:
        command  = ("DELETE FROM owned WHERE stock_id = '"+str(x)+"'")
        mycursor.execute(command)
        mycursor.close()
    except mysql.connector.Error as e:
        print("An error occured: {}".format(e))
    finally:
        if (mydb.is_connected()):
            mydb.close()
#assign_PID (Assign Purchase ID) - creates a primary key when inputing a purchase into the database.             
def assign_PID(k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    command = ("SELECT MAX(purchase_ID) FROM owned")
    mycursor.execute(command)
    x = mycursor.fetchall()[0][0]
    if x is None:
        x=0
        return x
    x=x+1
    return x
    

In [11]:
#Defining Functions
#checkkey - checks to see if a SID exists in the database to return true or false. Nested into other functions. 

def checkKey(ticker,o={}):
    x= get_SID(ticker,sql)
    for key in o:
        if x == o[key]['stock_id']:
            return True
            break
    return False

#add_to_owned checks to see if the the entered stock is in the database. If not, uses previous functions to allow the user to enter information into the database
# if the stock exists in the data, the user is prompted to update the infomation already stored in the database. Includes data validation and error handling. 
def add_to_owned():
    i=True
    while i != 'NO':
        owned = convert_to_owned(sql)
        show_table("owned",sql)
        print("""\nEnter a Ticker, or Press "Enter" to go back.""")
        t = input("\nTicker: ").upper()
        if t == "":
            break
        y = checkKey(t,owned )
        j=False
        if y is True:
            while j !='':
                j = input("""\nThat stock is already in the dictionary. Do you want to:
\n1) Add a new purchase instance.
2) Update Exsisting Information.
\n Press "Enter" to go back. \n""")
                if j == "1":
                    break
                elif j == "2":
                    update_owned(t,owned)
                    print("\nStock {} successfully updated!".format(t))
                    break
                elif j=="":
                    print("\nNo changes were made")
                    break
                else:
                    "Please enter a valid option"
        if y is False or j == "1":
            w=False
            z = {}
            z.update({'buy_date': ud_date()})
            z.update({'shares' : ud_share()})
            z.update({'cost': ud_cost()})
            z.update({'portfolio':ud_port()})
            print(NL,z)
            w = input("Do you want to commit these changes?\n").upper()
            if w == "YES":
                owned.update({t:z})
                insert_3("owned",t,z,sql)
                if InsertError != True:
                    print("\nStock {} successfully added!".format(t))      
            elif w == "NO":
                continue    
        i = input("\nDo you want to add/update another stock? ").upper()
        if i == "YES":
            continue
        else: 
            break

In [12]:
#Defining Functions
# show_table - queries the database and displays a html table with the information requested from outer function. 

def show_table(x,k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    try:
        quack = pd.read_sql('SELECT * FROM '+str(x), mydb)
        display(HTML(quack.to_html()))
    except mysql.connector.Error as e:
        print("An error occured: {}".format(e))
    finally:
        if (mydb.is_connected()):
            mydb.close()
            
#show_partial - queries the database and displays an html table with the a conidtional filter. 
def show_partial(x,y,t,k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    try:
        quack = pd.read_sql('SELECT * FROM '+str(t)+' WHERE '+str(x)+' = '+str(y), mydb)
        display(HTML(quack.to_html()))
    except mysql.connector.Error as e:
        print("An error occured: {}".format(e))
    finally:
        if (mydb.is_connected()):
            mydb.close()

In [13]:
#Defining functions
# check_for_dupes - checks to make sure that there are no duplicate values in the database before prompting the user to input 

def check_for_dupes(x,table,k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    command  = ("SELECT COUNT(stock_id) FROM " +str(table)+" WHERE stock_id = "+str(x))
    try:
        mycursor.execute(command)
        y =  mycursor.fetchall()[0][0]
        print(y)
        mycursor.close()
        sql_error=False
    except mysql.connector.Error as e:
        sql_error = True
        print("An error occured: {}".format(e))
    finally:
        if (mydb.is_connected()):
            mydb.close()
#get_tables - lists the tables currently in the databsae
def get_tables(k={}):
    mydb = mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    mycursor.execute("SHOW TABLES")
    tables = [item[0] for item in mycursor.fetchall()]
    return tables
#get_count - counts of the purchase ID values in the databse to show the number of owned stocks. 
def get_count(table,k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    command  = ("SELECT COUNT(purchase_id) FROM " +str(table))
    try:
        mycursor.execute(command)
        y =  mycursor.fetchall()[0][0]
        print(y)
        return y
        mycursor.close()
        sql_error=False
    except mysql.connector.Error as e:
        sql_error = True
        print("An error occured: {}".format(e))
    finally:
        if (mydb.is_connected()):
            mydb.close()
            

In [14]:
#convert_to_owned - Converts a dataframe to a format that can be input into a SQL statement. 
def convert_to_owned(k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    quack = pd.read_sql('SELECT * FROM owned', mydb)
    x = quack.set_index('purchase_id').T.to_dict('dict')
    o = {}
    for key,value in x.items():
        v = value
        o.update({key:v})
    return o 


In [15]:
#owned_check - checks the owned table to confirm that the stock is in the owned table.

def owned_check(x,k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    command = ("Select * FROM owned WHERE stock_ID = "+str(x))
    mycursor.execute(command)
    y = len(mycursor.fetchall())
    return y

#delete_from_db - deletes selected data from the database by row. 
def delete_from_db(t,k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    x = get_SID(t,sql)
    command  = ("DELETE FROM owned WHERE stock_id = "+str(x))
    try:
        sql_error =False
        mycursor.execute(command)
        mycursor.close()
        mydb.commit()
    except mysql.connector.Error as e:
        sql_error=True
        print("An error occured: {}".format(e))
    finally:
        if (mydb.is_connected()):
            mydb.close() 

#delete_from_db2 - same as delete from db, but prompts the user to select which stock that to delete. 
def delete_from_db2(t,k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    x = get_SID(t,sql)
    show_partial("stock_id",x,"owned",sql)
    i = input("Which do you want to delete?\n(Enter the Purchase ID #)\n")
    command  = ("DELETE FROM owned WHERE purchase_id = "+str(i))
    try:
        sql_error =False
        mycursor.execute(command)
        mycursor.close()
        mydb.commit()
    except mysql.connector.Error as e:
        sql_error=True
        print("An error occured: {}".format(e))
    finally:
        if (mydb.is_connected()):
            mydb.close()
#delete_Stock - uses user input to determine which stock to delete. 
def delete_stock():
    i=True
    q = True
    while i != "NO" or q !="":
        owned = convert_to_owned(sql)
        show_table("owned",sql)
        print(NL)
        print("""\tWhat stock do you want to delete? 
\tPress Enter to go back.\n""")
        q = input("Ticker: ").upper()
        if q == "":
            break
        else:
            t = get_SID(q,sql)
            x = owned_check(t,sql)
            if x == 0:
                print("That stock is not in the owned table.")
            if x == 1:
                delete_from_db(q,sql)
                print("Stock {} successfully deleted! ".format(q))
            if x > 1:
                delete_from_db2(q,sql)
                print("Stock {} successfully deleted! ".format(q))             
        i = input("Do you want to delete another stock? ").upper()

In [16]:
#run - Starts the program. Contains the code to start the program loop and prompt user for the intital input. 

def run(k={}):
    mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
    mycursor = cursor(mydb)
    global sql_error
    sql_error = False
    i = True
    while i !='EXIT':
        mydb=connect(k.get("a_p"), k.get("h"), k.get("u"), k.get("p"), k.get("db"))
        mycursor = cursor(mydb)
        tables= get_tables(sql)
        i=input("""What would you like to do?\n
                  1) Show me a table.
                  2) Input new stocks/update stocks.
                  3) Remove stocks you've sold.
                  Type "Exit" to exit the program.\n 
                  """
                   ).upper()
            
        if i == str(1):
            while i != "back":
                for x in tables:
                    print(x)
                i=input("""What table would you like to see?
Type "back" to go back.\n\n""").lower()
                if i in tables:
                    show_table(i,sql)
                elif i == "back":
                    break
                else:
                    print("Please enter a table from the list")
        elif i == str(2):
            
            add_to_owned()
        elif i == str(3):
            delete_stock()
        elif i == "EXIT":
            break
        else:
            print("That is not an option. Please enter 1, 2, 3, or Exit to end the program. ")
        


In [18]:
run(sql)

What would you like to do?

                  1) Show me a table.
                  2) Input new stocks/update stocks.
                  3) Remove stocks you've sold.
                  Type "Exit" to exit the program.
 
                  2


,purchase_id,stock_id,buy_date,shares,cost,portfolio
0,0,3184,2019-10-25,4.0,100.58,Robinhood
1,1,3729,2018-10-16,3.0,109.54,Robinhood
2,2,4444,2019-11-13,5.0,27.99,Robinhood
3,3,3962,2019-11-04,1.0,206.82,Robinhood
4,4,4868,2019-10-01,3.0,86.77,Robinhood
5,6,3729,2019-06-04,3.0,122.54,Robinhood
6,7,5104,2019-04-29,4.0,10.85,Robinhood
7,8,6162,2019-10-01,22.0,5.30,Robinhood
8,9,6162,2019-10-02,20.0,5.27,Robinhood
9,10,1396,2019-10-01,3.0,47.86,Robinhood



Enter a Ticker, or Press "Enter" to go back.

Ticker: sono

That stock is already in the dictionary. Do you want to:

1) Add a new purchase instance.
2) Update Exsisting Information.

 Press "Enter" to go back. 
1
What was the purchase date? (YY/MM/DD): 18/12/24
Number of shares purchased?: 5
What is the cost per share?: 9.46
Which portfolio is this stock in?
1) Vanguard
2) Robinhood
2

 {'buy_date': '18/12/24', 'shares': '5', 'cost': '9.46', 'portfolio': 'Robinhood'}
Do you want to commit these changes?
yes
INSERT INTO owned VALUES (13,5104,"18/12/24",5,9.46,"Robinhood")

Stock SONO successfully added!

Do you want to add/update another stock? no
What would you like to do?

                  1) Show me a table.
                  2) Input new stocks/update stocks.
                  3) Remove stocks you've sold.
                  Type "Exit" to exit the program.
 
                  exit
